In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
plt.style.use('ggplot')
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
import matplotlib.style as style

from sklearn.metrics import precision_score

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import partial_dependence, plot_partial_dependence
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

import textwrap # for wrapping answer descriptions
# from individual_soln import * # use functions from individual_soln.py

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/henryhoenig/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [3]:
df = pd.read_stata('data/anes_timeseries_2020_stata_20210324.dta')

In [7]:
counter = 0
for i in range(len(df)):
    if df['V202326'].iloc[i] == '3. Neither approve nor disapprove' and (df['V202073'].iloc[i] == '1. Joe Biden' or df['V201029'].iloc[i] == '1. Joe Biden'):
        counter +=1 
print(counter)

438


In [13]:
df['V202326'].value_counts('1. Approve')

1. Approve                                                        0.439855
2. Disapprove                                                     0.279348
3. Neither approve nor disapprove                                 0.173551
-6. No post-election interview                                    0.090580
-7. No post-election data, deleted due to incomplete interview    0.009300
-5. Interview breakoff (sufficient partial IW)                    0.006159
-9. Refused                                                       0.000725
-8. Don't know                                                    0.000483
Name: V202326, dtype: float64

In [ ]:
# 178 people approved of ACA but voted for Trump
# 529 were Neither approve nor disapprove and voted for Trump
# 118 disapproved and voted for Biden
# 438 neither approved nor disapproved and voted Biden

# 7396 people offered response (incl don't know, neither)
# 44% - 3642 people said they approved
# Only 4.9% of those who approved voted for Trump
# 27% disapproved - 2313 people
# 5.1% of those who disapproved voted for Biden

# 1. Approve                                                        3642
# 2. Disapprove                                                     2313
# 3. Neither approve nor disapprove                                 1437
# -6. No post-election interview                                    750 
# -7. No post-election data, deleted due to incomplete interview    77  
# -5. Interview breakoff (sufficient partial IW)                    51  
# -9. Refused                                                       6   
# -8. Don't know                                                    4   
# Name: V202326, dtype: int64

In [17]:
counter = 0
for i in range(len(df)):
    if df['V201384'].iloc[i] == '1. Favor' and (df['V202073'].iloc[i] == '2. Donald Trump' or df['V201029'].iloc[i] == '2. Donald Trump'):
        counter +=1 
print(counter)

54


In [15]:
df['V201384'].value_counts()

1. Favor                       3725
2. Oppose                      3301
3. Neither favor nor oppose    1213
-9. Refused                    36  
-8. Don't know                 5   
Name: V201384, dtype: int64

In [ ]:
# df['V201384'].value_counts()
# 1. Favor                       3725
# 2. Oppose                      3301
# 3. Neither favor nor oppose    1213
# -9. Refused                    36  
# -8. Don't know                 5   
# Name: V201384, dtype: int64
        
        
# 2923 favored House impeachment and voted for Biden (78% of those who favored)
# 212 opposed House impeachment and voted for Biden (6.4% of those who opposed)
# 54 people favored House impeachment and voted for Trump (1.4% of those who favored)

In [23]:
counter = 0
for i in range(len(df)):
    if df['V201387'].iloc[i] == '1. Favor' and (df['V202073'].iloc[i] == '1. Joe Biden' or df['V201029'].iloc[i] == '1. Joe Biden'):
        counter +=1 
print(counter)

168


In [20]:
df['V201387'].value_counts()

2. Oppose                      3699
1. Favor                       3113
3. Neither favor nor oppose    1422
-9. Refused                    40  
-8. Don't know                 6   
Name: V201387, dtype: int64

In [24]:
# 2. Oppose                      3699
# 1. Favor                       3113
# 3. Neither favor nor oppose    1422
# -9. Refused                    40  
# -8. Don't know                 6   
# Name: V201387, dtype: int64
        
# 2217 favored Senate acquittal and voted for Trump (71% of those who favored)
# 96 opposed Senate acquittal and voted for Trump (2.6%)
# 2869 opposed Senate acquittal and voted for Biden (77%)
# 168 favored acquittal and voted for Biden (5.3%)

In [29]:
counter = 0
for i in range(len(df)):
    if df['V201424'].iloc[i] == '2. Oppose' and (df['V202073'].iloc[i] == '2. Donald Trump' or df['V201029'].iloc[i] == '2. Donald Trump'):
        counter +=1 
print(counter)

142


In [26]:
df['V201424'].value_counts()

2. Oppose                      3788
1. Favor                       2983
3. Neither favor nor oppose    1472
-9. Refused                    32  
-8. Don't know                 5   
Name: V201424, dtype: int64

In [ ]:
# # 2. Oppose                      3788
# # 1. Favor                       2983
# # 3. Neither favor nor oppose    1472
# # -9. Refused                    32  
# # -8. Don't know                 5   
# # Name: V201424, dtype: int64
        

# 224 people who favored border wall voted Biden (7.5% of those who favored)
# 2779 who oppose voted Biden (73%)
# only 142 who opposed border wall voted for Trump (3.7%)

In [30]:
counter = 0
for i in range(len(df)):
    if df['V202527'].iloc[i] == '5. None at all' and (df['V202073'].iloc[i] == '2. Donald Trump' or df['V201029'].iloc[i] == '2. Donald Trump'):
        counter +=1 
print(counter)

123


In [31]:
df['V202527'].value_counts()

3. A moderate amount                                              2081
2. A lot                                                          2069
1. A great deal                                                   1860
4. A little                                                       1086
-6. No post-election interview                                    750 
5. None at all                                                    188 
-5. Interview breakoff (sufficient partial IW)                    97  
-7. No post-election data, deleted due to incomplete interview    77  
-9. Refused                                                       72  
Name: V202527, dtype: int64

In [ ]:
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(diabetes.feature_names)[sorted_idx])